In [1]:
import sys
sys.path.append('..')
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer, seed_everything
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split

from torch_explain.models.explainer import MuExplainer
from experiments.data.load_datasets import load_celldiff

In [2]:
x, y, concept_names = load_celldiff()

dataset = TensorDataset(x, y)

train_size = int(len(dataset) * 0.5)
val_size = (len(dataset) - train_size) // 2
test_size = len(dataset) - train_size - val_size
train_data, val_data, test_data = random_split(dataset, [train_size, val_size, test_size])
train_loader = DataLoader(train_data, batch_size=train_size)
val_loader = DataLoader(val_data, batch_size=val_size)
test_loader = DataLoader(test_data, batch_size=test_size)

n_concepts = next(iter(train_loader))[0].shape[1]
n_classes = 5

print(concept_names)
print(n_concepts)
print(n_classes)

Index(['alas2', 'si:xx-by187g17.1', 'si:ch211-5k11.6', 'tspan36', 'ba1l',
       'mmp9', 'il21r.1', 'plekhf1', 'slc3a2a', 'si:dkey-222f2.7',
       ...
       'si:ch211-195b11.3', 'apln', 'sele', 'thbs1b', 'zgc:171686', 'suv39h1b',
       'itga2b', 'tspan7', 'fn1b', 'tuba8l3'],
      dtype='object', length=1845)
1845
5


In [6]:
seed_everything(42)

base_dir = f'./results/celldiff/explainer'
os.makedirs(base_dir, exist_ok=True)

n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
results_list = []
for split, (trainval_index, test_index) in enumerate(skf.split(x.cpu().detach().numpy(), y.argmax(dim=1).cpu().detach().numpy())):
    print(f'Split [{split + 1}/{n_splits}]')
    x_trainval, x_test = torch.FloatTensor(x[trainval_index]), torch.FloatTensor(x[test_index])
    y_trainval, y_test = torch.FloatTensor(y[trainval_index]), torch.FloatTensor(y[test_index])
    x_train, x_val, y_train, y_val = train_test_split(x_trainval, y_trainval, test_size=0.2, random_state=42)
    print(f'{len(y_train)}/{len(y_val)}/{len(y_test)}')

    train_data = TensorDataset(x_train, y_train)
    val_data = TensorDataset(x_val, y_val)
    test_data = TensorDataset(x_test, y_test)
    train_loader = DataLoader(train_data, batch_size=train_size)
    val_loader = DataLoader(val_data, batch_size=val_size)
    test_loader = DataLoader(test_data, batch_size=test_size)

    checkpoint_callback = ModelCheckpoint(dirpath=base_dir, monitor='val_loss', save_top_k=1)
    trainer = Trainer(min_epochs=200, max_epochs=200, gpus=1, auto_lr_find=True, deterministic=True,
                      check_val_every_n_epoch=1, default_root_dir=base_dir,
                      weights_save_path=base_dir, callbacks=[checkpoint_callback])
    model = MuExplainer(n_concepts=n_concepts, n_classes=n_classes, l1=0.0001, lr=0.001, explainer_hidden=[30, 20, 10])

    trainer.fit(model, train_loader, val_loader)
    model.freeze()
    model_results = trainer.test(model, test_dataloaders=test_loader)
    results, results_full = model.explain_class(val_loader, test_loader, target_class='all',
                                  topk_explanations=1, max_minterm_complexity=20,
                                  concept_names=concept_names)
    results['model_accuracy'] = model_results[0]['test_acc']
    print(f"Explanation: {results_full[1]['explanation']}")
    print(f"Explanation: {results['explanation_accuracy']}")

    results_list.append(results)
#     break

Global seed set to 42
c:\users\pietr\anaconda3\envs\deep-logic\lib\site-packages\pytorch_lightning\utilities\distributed.py:52: UserWarning: Checkpoint directory ./results/celldiff/explainer exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | loss  | NLLLoss    | 0     
1 | model | Sequential | 281 K 
-------------------------------------
281 K     Trainable params
0         Non-trainable params
281 K     Total params
1.124     Total estimated model params size (MB)


Split [1/5]
909/228/285


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.9507042169570923, 'test_acc_epoch': 0.9507042169570923}
--------------------------------------------------------------------------------
Explanation: prune2
Explanation: 0.8964788732394366


In [7]:
results_df = pd.DataFrame(results_list)
print(results_df)
print(results_df.mean())

   explanation_accuracy  explanation_fidelity  explanation_complexity  \
0              0.842105              0.889123                     1.0   
1              0.891228              0.861053                     1.0   
2              0.861268              0.911972                    12.4   
3              0.919014              0.828873                     4.8   
4              0.896479              0.790845                     1.2   

   model_accuracy  
0        0.961404  
1        0.961404  
2        0.982394  
3        0.957746  
4        0.950704  
explanation_accuracy      0.882019
explanation_fidelity      0.856373
explanation_complexity    4.080000
model_accuracy            0.962730
dtype: float64


In [8]:
dt_scores, rf_scores = [], []
for split, (trainval_index, test_index) in enumerate(skf.split(x.cpu().detach().numpy(), y.argmax(dim=1).cpu().detach().numpy())):
    print(f'Split [{split + 1}/{n_splits}]')
    x_trainval, x_test = x[trainval_index], x[test_index]
    y_trainval, y_test = y[trainval_index].argmax(dim=1), y[test_index].argmax(dim=1)

    dt_model = DecisionTreeClassifier(max_depth=9, random_state=split)
    dt_model.fit(x_trainval, y_trainval)
    dt_scores.append(dt_model.score(x_test, y_test))

    rf_model = RandomForestClassifier(random_state=split)
    rf_model.fit(x_trainval, y_trainval)
    rf_scores.append(rf_model.score(x_test, y_test))

print(f'Random forest scores: {np.mean(rf_scores)} (+/- {np.std(rf_scores)})')
print(f'Decision tree scores: {np.mean(dt_scores)} (+/- {np.std(dt_scores)})')
print(f'Mu net scores (model): {results_df["model_accuracy"].mean()} (+/- {results_df["model_accuracy"].std()})')
print(f'Mu net scores (exp): {results_df["explanation_accuracy"].mean()} (+/- {results_df["explanation_accuracy"].std()})')

Split [1/5]
Split [2/5]
Split [3/5]
Split [4/5]
Split [5/5]
Random forest scores: 0.9613219668890537 (+/- 0.008900269829333018)
Decision tree scores: 0.9156066221892761 (+/- 0.0055169619258114986)
Mu net scores (model): 0.9627304196357727 (+/- 0.011828571209660013)
Mu net scores (exp): 0.882018779342723 (+/- 0.030357280600700885)
